In [1]:
import os
import pandas as pd
import numpy as np
from scipy.interpolate import interpn
from skimage.metrics import structural_similarity as ssim

from utils import AirDataManager


data_directory = "E:\TFG\core_dataset_sat\\air_dataset"
    

In [2]:
def bicubic_interpolation(image, objective_dim): # De momento lo implementare para 1 sola foto a la vez
    #Calculo nuevas dimensiones
    height, width = image.shape[0] , image.shape[1]
    new_width, new_height = objective_dim[0], objective_dim[1]
    new_image = np.zeros((new_height, new_width, image.shape[2]))
    
     # Generar cuadrículas para las coordenadas X e Y de la imagen original y la interpolada
    x = np.linspace(0, width - 1, width)
    y = np.linspace(0, height - 1, height)
    new_x = np.linspace(0, width - 1, new_width)
    new_y = np.linspace(0, height - 1, new_height)
    print("Interpolating....")
    new_image = interpn((y, x), image, (new_y[:,None], new_x), method='cubic', bounds_error=False, fill_value=0)
    print("--------------------")
    return new_image

def calculate_psnr(Hr, upscaled): 
    mse = np.mean((Hr - upscaled) ** 2) 
    if mse == 0:  # MSE is zero means no noise is present in the signal . 
                  # Therefore PSNR have no importance. 
        return 100
    max_pixel = 255.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse)) 
    return psnr 

def calculate_ssim(reference_image, comparison_image):
    # Ensure images have the same data type
    reference_image = reference_image.astype(np.float64)
    comparison_image = comparison_image.astype(np.float64)
    
    # Calculate SSIM
    ssim_index, _ = ssim(reference_image, comparison_image, full=True)
    
    return ssim_index

In [3]:
folders = os.listdir(data_directory)
folders = folders[:-1]
folders = sorted(list(map(int, folders)))
hr_datasets = folders[1:]
lr_datasets = folders[:-1]

In [4]:
experiments = []
sample_size = 500
results = {}
for hr,lr in zip(hr_datasets, lr_datasets):
    experiments.append(str(hr) + "," + str(lr))
    print(experiments[-1])
    
    manager = AirDataManager.AirDataManager(data_directory=data_directory, hr_dataset_name=str(hr), lr_dataset_name=str(lr))
    hr_data, lr_data = manager.get_random_data(n_samples=sample_size)
    experiment_results = {'psnr':[], 'ssim':[]}
    
    for index in range(sample_size):
        interpolated = bicubic_interpolation(lr_data[index].numpy(), (hr,hr))
        psnr = calculate_psnr(hr_data[index].numpy(), interpolated)
        ssim = calculate_ssim(hr_data[index].numpy(), interpolated)
        experiment_results['psnr'].append(psnr)
        experiment_results['ssim'].append(ssim)
        
    results[experiments[-1]] = (np.mean(experiment_results['psnr'], np.mean(experiment_results['ssim'])))
    print(results[-1])

64,32
Interpolating....
--------------------


ValueError: win_size exceeds image extent. Either ensure that your images are at least 7x7; or pass win_size explicitly in the function call, with an odd value less than or equal to the smaller side of your images. If your images are multichannel (with color channels), set channel_axis to the axis number corresponding to the channels.